In [35]:
import json
import re
from typing import Optional

import pandas as pd

In [52]:
df = pd.concat([
    pd.read_csv('./data/llm_perf/hf-dgx-01_perf-report.csv'),
    pd.read_csv('./data/llm_perf/audace_perf-report.csv')
])

In [53]:
df['gpu'] = df['environment.gpus'].apply(lambda x: json.loads(x.replace('\'', '"'))[0])

In [54]:
def parse_model_parameters_count(model_name: str) -> Optional[tuple[float, str]]:
    match = re.search(r'([0-9.]+)(b|m)', model_name, re.IGNORECASE)
    if match is not None:
        count, unit = float(match[1]), match[2].lower()
        if unit == 'm':
            count /= 1000
        return count

In [55]:
df['parameters_count'] = df['model'].apply(parse_model_parameters_count)

In [56]:
df['energy_per_token'] = df['generate.energy_consumption(kWh/token)'] * 1000

In [57]:
sub = df[[
    'model',
    'parameters_count',
    'gpu',
    'energy_per_token'
]].dropna()



In [58]:
len(sub)

1692

In [60]:
sub.to_csv('./data/aggregated_llm_perf.csv', index=False)